## Dependencies

In [81]:
!pip install pydub SpeechRecognition word2number
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [82]:
from google.colab import output
import re
from word2number import w2n
import nltk
from nltk import word_tokenize, pos_tag, sent_tokenize

In [83]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [84]:
# Define a global variable to store the transcript
transcript_text = ""

In [85]:
# Function to handle uploaded audio
def upload_audio(audio_data):
    global transcript_text
    file_path = decode_audio(audio_data)
    transcript_text = transcribe_audio(file_path)
    transcript_text = punctuate_text(transcript_text)
    print(f"Transcription: {transcript_text}")

output.register_callback('notebook.upload_audio', upload_audio)

In [86]:
from IPython.display import display, Javascript
import base64
import speech_recognition as sr
from pydub import AudioSegment
import io

## Record Audio Prompt

**Use Prompt Format:** I need to create a table that has a height of [----] feet , with a thickness of [----] inches, with [----] sides, and a radius of [----] feet. The table must have a corner fillet of [----] inches, with overall edge fillets of [----] inches. For the legs, make the leg radius [----] inches, with a skew of [----] degrees and a taper of [----] inches.

In [189]:
# JavaScript code for recording audio
record_audio_js = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))

const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})

let recorder, stream, chunks = []

async function startRecording() {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  start_button.disabled = true
  stop_button.disabled = false
}

function stopRecording() {
  return new Promise(async resolve => {
    recorder.onstop = async () => {
      const blob = new Blob(chunks)
      const audio = await b2text(blob)
      resolve(audio)
    }
    recorder.stop()
    stream.getTracks().forEach(track => track.stop())
    start_button.disabled = false
    stop_button.disabled = true
  })
}

var start_button = document.createElement('button');
start_button.textContent = "Start Recording";
start_button.style.display = "block";
start_button.style.margin = "10px 0";
start_button.onclick = startRecording;

var stop_button = document.createElement('button');
stop_button.textContent = "Stop Recording";
stop_button.style.display = "block";
stop_button.style.margin = "10px 0";
stop_button.disabled = true;
stop_button.onclick = async () => {
  const audio = await stopRecording();
  google.colab.kernel.invokeFunction('notebook.upload_audio', [audio], {});
};

document.body.appendChild(start_button);
document.body.appendChild(stop_button);
"""

display(Javascript(record_audio_js))

def decode_audio(audio_data):
    audio_data = audio_data.split(',')[1]
    audio_bytes = base64.b64decode(audio_data)
    audio = AudioSegment.from_file(io.BytesIO(audio_bytes), format="webm")
    audio.export("output.wav", format="wav")
    return "output.wav"

# Function to transcribe audio
def transcribe_audio(file_path):
    recognizer = sr.Recognizer()
    with sr.AudioFile(file_path) as source:
        audio = recognizer.record(source)
    try:
        return recognizer.recognize_google(audio)
    except sr.UnknownValueError:
        return "Could not understand audio"
    except sr.RequestError as e:
        return f"Could not request results; {e}"

# Function to punctuate text
def punctuate_text(text):
    # Split text into sentences
    sentences = sent_tokenize(text)
    punctuated_sentences = []

    for sentence in sentences:
        # Capitalize the first letter of the sentence
        sentence = sentence.capitalize()

        # Convert numbers from words to digits
        sentence = re.sub(r'\b(one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|twenty)\b',
                          lambda match: str(w2n.word_to_num(match.group())), sentence)

        # Add commas before conjunctions if they follow a clause
        sentence = re.sub(r'(\b\w+\b)\s(and|but|or|nor|for|so|yet)\s(\b\w+\b)',
                          r'\1, \2 \3', sentence)

        # Add commas in lists (basic approach for commas in lists)
        sentence = re.sub(r'(\b\w+\b)\s(\b\w+\b)\s(and|or)\s(\b\w+\b)',
                          r'\1, \2, \3 \4', sentence)

        # Remove commas that follow numbers
        sentence = re.sub(r'(\d+),(\s)', r'\1\2', sentence)  # Remove comma directly after numbers

        # Ensure the sentence ends with a period
        if not sentence.endswith('.'):
            sentence += '.'

        punctuated_sentences.append(sentence)

    return ' '.join(punctuated_sentences)

print(transcript_text)


<IPython.core.display.Javascript object>

I need to create a table that has a height of 3.5 ft with a thickness of 0.25 in with 4 sides, and a radius of 2.5 ft.
Transcription: I need to create a table that has a height of 3.5 ft with a thickness of 0.25 in with 5 sides, and a radius of 5 ft the table must have a corner fillet of 0.41 inches, and overall edge fillets of 0.1 inches, for the legs make the leg radius 0.2 inches with a few of 1.25 degrees, and the taper of 0.25 inches.


**Use Prompt Format:** I need to create a table that has a height of [----] feet , with a thickness of [----] inches, with [----] sides, and a radius of [----] feet. The table must have a corner fillet of [----] inches, with overall edge fillets of [----] inches. For the legs, make the leg radius [----] inches, with a skew of [----] degrees and a taper of [----] inches.

In [190]:
prompt = transcript_text
print(prompt)

I need to create a table that has a height of 3.5 ft with a thickness of 0.25 in with 5 sides, and a radius of 5 ft the table must have a corner fillet of 0.41 inches, and overall edge fillets of 0.1 inches, for the legs make the leg radius 0.2 inches with a few of 1.25 degrees, and the taper of 0.25 inches.


## Capitalize / Fix Prompt

In [192]:
# List of specific words to capitalize
words_to_capitalize = ["tall", "height", "thickness", "radius", "corner fillet", "edge fillets", "leg radius", "skew", "taper", "degrees", "ft", "feet", "inch", "inches", "in"]

# Tokenize the prompt into words
tokens = word_tokenize(prompt)

# Perform part-of-speech tagging
tagged_tokens = pos_tag(tokens)

# Initialize a list to store capitalized words
capitalized_words = []

In [193]:
# Iterate through tagged tokens
for token, tag in tagged_tokens:
    if (token.lower() in words_to_capitalize or
        tag.startswith('NN')):  # NN for nouns, VB for verbs, JJ for adjectives
        capitalized_words.append(token.capitalize())
    else:
        capitalized_words.append(token)

# Join the words back into a single string
capitalized_prompt = ' '.join(capitalized_words)

# Remove any space before the period at the end
if capitalized_prompt.endswith('.'):
    capitalized_prompt = capitalized_prompt[:-2] + '.'

In [194]:
print("Original Prompt:")
print(prompt)
print("\nCapitalized Prompt:")
print(capitalized_prompt)

Original Prompt:
I need to create a table that has a height of 3.5 ft with a thickness of 0.25 in with 5 sides, and a radius of 5 ft the table must have a corner fillet of 0.41 inches, and overall edge fillets of 0.1 inches, for the legs make the leg radius 0.2 inches with a few of 1.25 degrees, and the taper of 0.25 inches.

Capitalized Prompt:
I need to create a Table that has a Height of 3.5 Ft with a Thickness of 0.25 In with 5 Sides , and a Radius of 5 Ft the Table must have a corner Fillet of 0.41 Inches , and overall Edge Fillets of 0.1 Inches , for the Legs make the Leg Radius 0.2 Inches with a few of 1.25 Degrees , and the Taper of 0.25 Inches.


## Find / Assign Variables for Family Member Creation

In [195]:
import re

In [222]:
def extract_variables(capitalized_prompt):
    # Define regular expressions for each variable
    height_pattern = r'Height\s+of\s+(\d+\.?\d*)\s+Ft'
    thickness_pattern = r'Thickness\s+of\s+(\d+\.?\d*)\s+In'
    sides_pattern = r'with\s+(\d+)\s+Sides'
    radius_pattern = r'Radius\s+of\s+(\d+\.?\d*)\s+Ft'
    cornerfillet_pattern = r'corner Fillet\s+of\s+(\d+\.?\d*)\s+Inches'
    edgefillet_pattern = r'Edge Fillets\s+of\s+(\d+\.?\d*)\s+Inches'
    legradius_pattern = r'Leg Radius\s+(\d+\.?\d*)\s+Inches'
    skew_pattern = r'Skew\s+of\s+(\d+\.?\d*)\s+Degrees'
    taper_pattern = r'Taper\s+of\s+(\d+\.?\d*)\s+Inches'

    # Search for each pattern in the prompt
    height_match = re.search(height_pattern, capitalized_prompt, re.IGNORECASE)
    thickness_match = re.search(thickness_pattern, capitalized_prompt, re.IGNORECASE)
    sides_match = re.search(sides_pattern, capitalized_prompt, re.IGNORECASE)
    radius_match = re.search(radius_pattern, capitalized_prompt, re.IGNORECASE)
    cornerfillet_match = re.search(cornerfillet_pattern, capitalized_prompt, re.IGNORECASE)
    edgefillet_match = re.search(edgefillet_pattern, capitalized_prompt, re.IGNORECASE)
    legradius_match = re.search(legradius_pattern, capitalized_prompt, re.IGNORECASE)
    skew_match = re.search(skew_pattern, capitalized_prompt, re.IGNORECASE)
    taper_match = re.search(taper_pattern, capitalized_prompt, re.IGNORECASE)

    # Extract values or return None if not found
    height = float(height_match.group(1)) if height_match else None
    thickness = float(thickness_match.group(1)) if thickness_match else None
    sides = int(sides_match.group(1)) if sides_match else None
    radius = float(radius_match.group(1)) if radius_match else None
    cornerfillet = float(cornerfillet_match.group(1)) if cornerfillet_match else None
    edgefillet = float(edgefillet_match.group(1)) if edgefillet_match else None
    legradius = float(legradius_match.group(1)) if legradius_match else None
    skew = float(skew_match.group(1)) if skew_match else None
    taper = float(taper_match.group(1)) if taper_match else None

    return {
        'Height': height,
        'Thickness': thickness,
        'Sides': sides,
        'Radius': radius,
        'Corner Fillet': cornerfillet,
        'Edge Fillets': edgefillet,
        'Leg Radius': legradius,
        'Skew': skew,
        'Taper': taper
    }

In [223]:
# Example usage
variables = extract_variables(capitalized_prompt)
print(variables)

{'Height': 3.5, 'Thickness': 0.25, 'Sides': 5, 'Radius': 5.0, 'Corner Fillet': 0.41, 'Edge Fillets': 0.1, 'Leg Radius': 0.2, 'Skew': None, 'Taper': 0.25}
